In [1]:
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

"""
The K-means algorithm written from scratch against PySpark. In practice,
one may prefer to use the KMeans algorithm in ML, as shown in
examples/src/main/python/ml/kmeans_example.py.

This example requires NumPy (http://www.numpy.org/).
"""
from __future__ import print_function

import sys

import numpy as np
import matplotlib.pyplot as plt
import time
from pyspark.sql import SparkSession

In [2]:
def parseVector(line):
    return np.array([float(x) for x in line.split(',')])

def closestPoint(p, centers):
    bestIndex = 0
    closest = float("+inf")
    for i in range(len(centers)):
        tempDist = np.sum((p - centers[i]) ** 2)
        if tempDist < closest:
            closest = tempDist
            bestIndex = i
    return bestIndex

def compute_error(p, centers):
    center = centers[closestPoint(p,centers)]
    return np.sqrt(sum([x**2 for x in (p - center)]))
    

In [5]:
spark = SparkSession\
    .builder\
    .appName("PythonKMeans")\
    .getOrCreate()

lines = spark.read.text('vsmallpointdata2018.txt').rdd.map(lambda r: r[0])
start_time = time.time()
data = lines.map(parseVector).cache()
convergeDist = 0.0001
tempDist = 1.0
i = 0
K = 5
kPoints = data.takeSample(False, K, int(time.time()))
while tempDist > convergeDist:
    closest = data.map(lambda p: (closestPoint(p, kPoints), (p, 1)))
    pointStats = closest.reduceByKey(lambda p1_c1, p2_c2:
                                     (p1_c1[0] + p2_c2[0], p1_c1[1] + p2_c2[1]))
    newPoints = pointStats.map(lambda st: (st[0], st[1][0] / st[1][1])).collect()
    tempDist = sum(np.sum((kPoints[iK] - p) ** 2) for (iK, p) in newPoints)
    ##updates new points coordinates
    for (iK, p) in newPoints:
        kPoints[iK] = p
WSSSE = data.map(lambda point: compute_error(point, kPoints)).reduce(lambda x, y: x + y)
elapsed_time = time.time() - start_time
print("Final centers: " + str(kPoints))
spark.stop()

Final centers: [array([96.1005216 , 95.55654445, 96.04460059, 95.43202245, 95.38561547]), array([102.50002344, 102.55730969, 102.54693349, 102.50004225,
       102.50375542]), array([12.93868093, 12.9427222 , 12.98174172, 12.96740589, 12.9358063 ]), array([52.46412729, 52.53837575, 52.46880548, 52.47931055, 52.50672933]), array([82.49692202, 82.50885243, 82.43371117, 82.50611784, 82.48918292])]
